<a href="https://colab.research.google.com/github/z3r0st/TF-201711448-20181C074-202021767/blob/main/Basauri_aporte_Hito3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def reescalado(val,rangeMin, rangeMax):
  rangeReal = rangeMax - rangeMin
  val += 1
  return rangeMin + ((rangeReal*val)/2.0)

In [ ]:
reescalado(-0.5,1,100)

25.75

In [8]:
import heapq as hq
import math

In [27]:
def __path(path, s, t):
  i = t
  route = [i]
  while i!=s:
    route.append(path[i])
    i = path[i]
    if path[i] == -1 and i != s:
      return []
  route.reverse()
  return route

In [28]:
def dijkstra(G, s, t = None):
  n = len(G)
  visited = [False]*n
  path = [-1]*n
  cost = [math.inf]*n

  cost[s] = 0
  pqueue = [(0, s)]
  while pqueue:
    g, u = hq.heappop(pqueue)
    if not visited[u]:
      visited[u] = True
      for v, w in G[u]:
        if not visited[v]:
          f = g + w
          if f < cost[v]:
            cost[v] = f
            path[v] = u
            hq.heappush(pqueue, (f, v))
  if t:
    return [__path(path, s, t), cost[t]]
  else: 
    return path, cost

In [43]:
def remove_edge(G, n_from, n_to):
  cost = None
  for intercept in G[n_from]:
    if intercept[0] == n_to:
      cost = intercept[1]
      if cost == math.inf:
        return -1
      else:
        intercept[1] = math.inf
        return cost

In [44]:
def append_edge(G, n_from, n_to, cost):
  for intercept in G[n_from]:
    if intercept[0] == n_to:
      intercept[1] = cost
      return

In [45]:
def YenKSP(G, init, end, K):
  global_path,costs = dijkstra(G, init)
  A = [{
      'cost': costs[end],
      'path': __path(global_path, init, end)
  }]
  B = []

  if A[0]['path'] == []: return A

  for k in range(1, K):
    for i in range(0, len(A[k-1]['path'])-1):
      spurNode = A[k -1]['path'][i]
      rootPath = A[k -1]['path'][:i+1]

      temp_removed = []
      for p in A:
        if rootPath == p['path'][:i+1]:
          cost = remove_edge(G,p['path'][i],p['path'][i+1])
          if cost == -1:
            continue
          temp_removed.append([p['path'][i],p['path'][i+1],cost])
      
      spurPath = dijkstra(G, spurNode, end)

      if spurPath != []:
        totalPath = rootPath[:-1] + spurPath[0]
        totalCost = costs[spurNode] + spurPath[1]
        candidate = {'cost': totalCost, 'path': totalPath}

        if not (candidate in B) and not (candidate in A):
          B.append(candidate)
      
      for temp in temp_removed:
        append_edge(G, temp[0], temp[1], temp[2])
    if len(B) == 0:
      break
    B = sorted(B, key = lambda d: d['cost'])
    A.append(B[0])
    B.pop(0)
  return A

#Testing

In [29]:
%%file 0502.al
1 2 3 2 4 9
0 2 2 1 3 2 5 2
1 1 4 2 6 2
0 2 1 2 5 9 7 3
0 9 2 2 7 8
1 2 3 9 6 7
2 2 5 7 7 3
3 3 4 8 6 3

Overwriting 0502.al


In [37]:
with open("0502.al") as f:
  G = []
  for line in f:
    nums = [int(x) for x in line.split()]
    G.append([])
    for i in range(0, len(nums), 2):
      G[-1].append([nums[i], nums[i+1]])

for x in G:
  print(x)

[[1, 2], [3, 2], [4, 9]]
[[0, 2], [2, 1], [3, 2], [5, 2]]
[[1, 1], [4, 2], [6, 2]]
[[0, 2], [1, 2], [5, 9], [7, 3]]
[[0, 9], [2, 2], [7, 8]]
[[1, 2], [3, 9], [6, 7]]
[[2, 2], [5, 7], [7, 3]]
[[3, 3], [4, 8], [6, 3]]


In [46]:
YenKSP(G,3,6,3)

[{'cost': 5, 'path': [3, 1, 2, 6]},
 {'cost': 6, 'path': [3, 7, 6]},
 {'cost': 7, 'path': [3, 0, 1, 2, 6]}]